# Project Proposal: Case study analysis: Holidays in COVID-19  #

Authors: Fares Burwag, Nikko Dumrique (63631204)

In [1]:
install.packages("conflicted")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(tidyverse)
library(repr)
library(datateachr)
library(digest)
library(infer)
library(gridExtra)
library(cowplot)
library(dplyr)
library(conflicted)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘infer’ was built under R version 4.0.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

 

In [3]:
conflict_prefer("select", "dplyr")
conflict_prefer("filter", "dplyr")

[conflicted] Will prefer dplyr::select over any other package

[conflicted] Will prefer dplyr::filter over any other package



## Data Wrangling ##

In [4]:
#reading public covid-19 data from https://health-infobase.canada.ca
covid <- read_csv("https://health-infobase.canada.ca/src/data/covidLive/covid19-download.csv")
head(covid)

Parsed with column specification:
cols(
  .default = col_double(),
  prname = col_character(),
  prnameFR = col_character(),
  date = col_date(format = ""),
  update = col_logical(),
  percentrecover = col_character()
)

See spec(...) for full column specifications.



pruid,prname,prnameFR,date,update,numconf,numprob,numdeaths,numtotal,numtested,⋯,ratedeaths_last14,numtotal_last7,ratetotal_last7,numdeaths_last7,ratedeaths_last7,avgtotal_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7,raterecovered
<dbl>,<chr>,<chr>,<date>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
35,Ontario,Ontario,2020-01-31,NA,3,0,0,3,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
59,British Columbia,Colombie-Britannique,2020-01-31,NA,1,0,0,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
1,Canada,Canada,2020-01-31,NA,4,0,0,4,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
35,Ontario,Ontario,2020-02-08,NA,3,0,0,3,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
59,British Columbia,Colombie-Britannique,2020-02-08,NA,4,0,0,4,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
1,Canada,Canada,2020-02-08,NA,7,0,0,7,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0


we would like to localize the datatset to canadian provinces and territories, therefore we will remove prnames that are not one of the provinces/territories. Additionally would like to remove the observations with numtested == NA. 

In [5]:
#filtering the dataset to get strictly provicincal observations in our dataset
provinces = c('Ontario','British Columbia','Quebec','Alberta',
              'Saskatchewan','Manitoba','New Brunswick','Newfoundland and Labrador',
              'Nova Scotia','Prince Edward Island','Northwest Territories','Nunavut','Yukon')

covid = covid  %>% 
    filter(prname %in% provinces)  %>% 
    filter(!is.na(numtested))
head(covid)

pruid,prname,prnameFR,date,update,numconf,numprob,numdeaths,numtotal,numtested,⋯,ratedeaths_last14,numtotal_last7,ratetotal_last7,numdeaths_last7,ratedeaths_last7,avgtotal_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7,raterecovered
<dbl>,<chr>,<chr>,<date>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
59,British Columbia,Colombie-Britannique,2020-03-11,NA,39,0,1,39,4373,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
48,Alberta,Alberta,2020-03-11,NA,14,0,0,14,1969,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
47,Saskatchewan,Saskatchewan,2020-03-11,NA,0,0,0,0,204,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
46,Manitoba,Manitoba,2020-03-11,NA,0,0,0,0,352,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
35,Ontario,Ontario,2020-03-11,NA,42,0,1,42,3394,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
24,Quebec,Québec,2020-03-11,NA,7,0,0,7,556,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0


In [56]:
#we will now select the columns we would like to work with
covid_selected = covid  %>%
    select(c(prname, date, numconf, numdeaths, numtested, numtoday,ratetotal))
head(covid_selected)

prname,date,numconf,numdeaths,numtested,numtoday,ratetotal
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
British Columbia,2020-03-11,39,1,4373,7,0.76
Alberta,2020-03-11,14,0,1969,7,0.32
Saskatchewan,2020-03-11,0,0,204,0,0.00
Manitoba,2020-03-11,0,0,352,0,0.00
Ontario,2020-03-11,42,1,3394,8,0.29
Quebec,2020-03-11,7,0,556,3,0.08


In [57]:
covid_selected$date <- as.Date(covid_wrangled$date)
head(covid_selected)

prname,date,numconf,numdeaths,numtested,numtoday,ratetotal
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
British Columbia,2020-03-11,39,1,4373,7,0.76
Alberta,2020-03-11,14,0,1969,7,0.32
Saskatchewan,2020-03-11,0,0,204,0,0.00
Manitoba,2020-03-11,0,0,352,0,0.00
Ontario,2020-03-11,42,1,3394,8,0.29
Quebec,2020-03-11,7,0,556,3,0.08


As we are comparing the timeline between confirmed cases to the holidays, we will categorize and filter our data according to canadian holidays

In [58]:
#we will use the University of Waterloo dataset to have a tibble of Canadian holidays
cdn_holidays  <- read_csv("https://raw.githubusercontent.com/uWaterloo/Datasets/master/Holidays/holidays.csv")
head(cdn_holidays)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  holiday = col_character()
)



date,holiday
<date>,<chr>
2012-01-02,New Year's Day
2012-02-20,Family Day
2012-04-06,Good Friday
2012-05-21,Victoria Day
2012-07-02,Canada Day
2012-08-06,Civic Holiday


In [9]:
#We will only consider Canadian public holidays within the date of the first observation 
# and last observation of the covid dataset

public_holidays  <- cdn_holidays  %>% 
    filter(holiday %in% c("New Year's Day", "Good Friday", "Canada Day", 
                          "Labour Day", "Thanksgiving", "Christmas Day"))  %>% 
    filter(date >= min(covid_wrangled$date), date <= max(covid_wrangled$date))
                    
public_holidays

date,holiday
<date>,<chr>
2020-04-10,Good Friday
2020-07-01,Canada Day
2020-09-07,Labour Day
2020-10-12,Thanksgiving
2020-12-25,Christmas Day
2021-01-01,New Year's Day


In [59]:
#as mentioned from cdc.gov, symptoms may appear 2-14 days after first encounter
#within the holidays dataframe, we would like to measure dates 2 weeks (14 days) after the holiday. 
#We will give a little room for celebrations celebrated close to the actual holiday (14 days will become 17 days).

#likewise we will also add a column for a week in advance to the holiday.
public_holiday_bound <- public_holidays  %>% 
    mutate(post_holiday = date + 17, pre_holiday = date - 7) 

public_holiday_bound

date,holiday,post_holiday,pre_holiday
<date>,<chr>,<date>,<date>
2020-04-10,Good Friday,2020-04-27,2020-04-03
2020-07-01,Canada Day,2020-07-18,2020-06-24
2020-09-07,Labour Day,2020-09-24,2020-08-31
2020-10-12,Thanksgiving,2020-10-29,2020-10-05
2020-12-25,Christmas Day,2021-01-11,2020-12-18
2021-01-01,New Year's Day,2021-01-18,2020-12-25


In [60]:
#holiday bounds
pre_good_friday = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == 'Good Friday'])
pre_canada_day = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == 'Canada Day'])
pre_labour_day = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == 'Labour Day'])
pre_thanksgiving = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == 'Thanksgiving'])
pre_christmas = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == 'Christmas Day'])
pre_new_years = ymd(public_holiday_bound$pre_holiday[public_holiday_bound$holiday == "New Year's Day"])

post_good_friday = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == 'Good Friday'])
post_canada_day = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == 'Canada Day'])
post_labour_day = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == 'Labour Day'])
post_thanksgiving = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == 'Thanksgiving'])
post_christmas = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == 'Christmas Day'])
post_new_years = ymd(public_holiday_bound$post_holiday[public_holiday_bound$holiday == "New Year's Day"])

good_friday = ymd(public_holiday_bound$date[public_holiday_bound$holiday == 'Good Friday'])
canada_day = ymd(public_holiday_bound$date[public_holiday_bound$holiday == 'Canada Day'])
labour_day = ymd(public_holiday_bound$date[public_holiday_bound$holiday == 'Labour Day'])
thanksgiving = ymd(public_holiday_bound$date[public_holiday_bound$holiday == 'Thanksgiving'])
christmas = ymd(public_holiday_bound$date[public_holiday_bound$holiday == 'Christmas Day'])
new_years = ymd(public_holiday_bound$date[public_holiday_bound$holiday == "New Year's Day"])

In [61]:
#here we are binding the observation to the given holiday period
covid_clean  <- covid_selected  %>%  
    mutate(holiday = case_when(
    (.$date >= pre_good_friday) & (.$date <= post_good_friday) ~ "Good Friday",
    (.$date >= pre_canada_day) & (.$date <= post_canada_day) ~ "Canada Day",
    (.$date >= pre_labour_day) & (.$date <= post_labour_day) ~ "Labour Day",
    (.$date >= pre_thanksgiving) & (.$date <= post_thanksgiving) ~ "Thanksgiving", 
    (.$date >= pre_christmas) & (.$date <= post_christmas) ~ "Christmas Day",
    (.$date >= pre_new_years) & (.$date <= post_new_years) ~ "New Year's Day"))  %>% 
    filter(!is.na(holiday))

head(covid_clean)

prname,date,numconf,numdeaths,numtested,numtoday,ratetotal,holiday
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
British Columbia,2020-04-03,1174,35,39926,53,22.81,Good Friday
Alberta,2020-04-03,1075,18,60508,107,24.31,Good Friday
Saskatchewan,2020-04-03,220,3,11720,14,18.66,Good Friday
Manitoba,2020-04-03,164,2,11650,15,13.20,Good Friday
Ontario,2020-04-03,3255,67,67998,462,22.09,Good Friday
Quebec,2020-04-03,6101,61,83570,583,71.15,Good Friday


additionally we will add a column to find the proportion between number of confirmed cases to the number of tested individuals 

In [62]:
#we add another category to identify if the observation is pre, post, or during the holiday
covid_categorized  <- covid_clean  %>%  
    mutate(category = case_when(
    (.$date >= pre_good_friday) & (.$date < good_friday)  | 
    (.$date >= pre_canada_day) & (.$date < canada_day) |
    (.$date >= pre_labour_day) & (.$date < labour_day) |
    (.$date >= pre_thanksgiving) & (.$date < thanksgiving) |
    (.$date >= pre_christmas) & (.$date < christmas) |
    (.$date >= pre_new_years) & (.$date < new_years) ~ "pre holiday",
    (.$date <= post_good_friday) & (.$date > good_friday)  | 
    (.$date <= post_canada_day) & (.$date > canada_day) |
    (.$date <= post_labour_day) & (.$date > labour_day) |
    (.$date <= post_thanksgiving) & (.$date > thanksgiving) |
    (.$date <= post_christmas) & (.$date > christmas) |
    (.$date <= post_new_years) & (.$date > new_years) ~ "post holiday",
    (.$date == good_friday)  | 
    (.$date == canada_day) |
    (.$date == labour_day) |
    (.$date == thanksgiving) |
    (.$date == christmas) |
    (.$date == new_years) ~ "during holiday"))
           
head(covid_categorized)

prname,date,numconf,numdeaths,numtested,numtoday,ratetotal,holiday,category
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
British Columbia,2020-04-03,1174,35,39926,53,22.81,Good Friday,pre holiday
Alberta,2020-04-03,1075,18,60508,107,24.31,Good Friday,pre holiday
Saskatchewan,2020-04-03,220,3,11720,14,18.66,Good Friday,pre holiday
Manitoba,2020-04-03,164,2,11650,15,13.20,Good Friday,pre holiday
Ontario,2020-04-03,3255,67,67998,462,22.09,Good Friday,pre holiday
Quebec,2020-04-03,6101,61,83570,583,71.15,Good Friday,pre holiday


In [63]:
covid_clean  <-  mutate(covid_categorized, propconfirmed = numconf / numtested)
head(covid_clean)

prname,date,numconf,numdeaths,numtested,numtoday,ratetotal,holiday,category,propconfirmed
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
British Columbia,2020-04-03,1174,35,39926,53,22.81,Good Friday,pre holiday,0.02940440
Alberta,2020-04-03,1075,18,60508,107,24.31,Good Friday,pre holiday,0.01776625
Saskatchewan,2020-04-03,220,3,11720,14,18.66,Good Friday,pre holiday,0.01877133
Manitoba,2020-04-03,164,2,11650,15,13.20,Good Friday,pre holiday,0.01407725
Ontario,2020-04-03,3255,67,67998,462,22.09,Good Friday,pre holiday,0.04786905
Quebec,2020-04-03,6101,61,83570,583,71.15,Good Friday,pre holiday,0.07300467
